In [1]:

%matplotlib inline

import sklearn
from sklearn.feature_selection import SelectKBest




import gc
import datetime as dt


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
station = pd.read_csv('station.csv', low_memory=False)
trip_test = pd.read_csv('trip_test.csv', low_memory=False)
trip_train = pd.read_csv('trip_train.csv', low_memory=False)
weather = pd.read_csv('weather.csv', low_memory=False)


In [42]:
trip_train = trip_test

In [43]:
trip_train = pd.merge(trip_train, station[['id', 'lat', 'long']], left_on='start_station_id', right_on= 'id' ,how='left')


In [44]:
trip_train.drop('id_y', axis=1, inplace=True)
trip_train.rename(columns={'id_x' : 'id', 'lat': 'start_station_lat','long': 'start_station_long'}, inplace=True)

In [45]:
trip_train = pd.merge(trip_train, station[['id', 'lat', 'long']], left_on='end_station_id', right_on= 'id' ,how='left')
trip_train

,id_x,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,id_y,lat,long
0,504737,10/18/2014 11:25,Embarcadero at Sansome,60,10/18/2014 12:12,Powell at Post (Union Square),71,426,Customer,77009,37.804770,-122.403234,71,37.788446,-122.408499
1,530846,11/5/2014 13:00,Embarcadero at Folsom,51,11/5/2014 13:09,Broadway St at Battery St,82,454,Subscriber,94132,37.791464,-122.391034,82,37.798541,-122.400862
2,813140,6/18/2015 17:34,San Francisco Caltrain (Townsend at 4th),70,6/18/2015 17:37,2nd at Townsend,61,370,Subscriber,94107,37.776617,-122.395260,61,37.780526,-122.390288
3,897674,8/20/2015 7:06,Civic Center BART (7th at Market),72,8/20/2015 7:15,Townsend at 7th,65,451,Subscriber,94582,37.781039,-122.411748,65,37.771058,-122.402717
4,322830,6/13/2014 8:46,San Francisco Caltrain 2 (330 Townsend),69,6/13/2014 8:57,Embarcadero at Folsom,51,603,Subscriber,95014,37.776600,-122.395470,51,37.791464,-122.391034
5,487841,10/7/2014 21:41,2nd at Townsend,61,10/7/2014 21:47,Post at Kearny,47,478,Subscriber,94115,37.780526,-122.390288,47,37.788975,-122.403452
6,677808,3/11/2015 18:09,Market at 10th,67,3/11/2015 18:22,San Francisco Caltrain (Townsend at 4th),70,505,Subscriber,94025,37.776619,-122.417385,70,37.776617,-122.395260
7,704449,3/30/2015 17:29,Embarcadero at Vallejo,48,3/30/2015 17:35,Steuart at Market,74,356,Subscriber,94536,37.799953,-122.398525,74,37.794139,-122.394434
8,833587,7/5/2015 11:54,Market at 10th,67,7/5/2015 12:00,Market at 4th,76,401,Subscriber,94102,37.776619,-122.417385,76,37.786305,-122.404966
9,420411,8/22/2014 13:30,Embarcadero at Vallejo,48,8/22/2014 13:41,5th at Howard,57,363,Subscriber,94114,37.799953,-122.398525,57,37.781752,-122.405127


In [46]:
trip_train.drop('id_y', axis=1, inplace=True)
trip_train.rename(columns={'id_x' : 'id', 'lat': 'end_station_lat','long': 'end_station_long'}, inplace=True)
trip_train

,id,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,end_station_lat,end_station_long
0,504737,10/18/2014 11:25,Embarcadero at Sansome,60,10/18/2014 12:12,Powell at Post (Union Square),71,426,Customer,77009,37.804770,-122.403234,37.788446,-122.408499
1,530846,11/5/2014 13:00,Embarcadero at Folsom,51,11/5/2014 13:09,Broadway St at Battery St,82,454,Subscriber,94132,37.791464,-122.391034,37.798541,-122.400862
2,813140,6/18/2015 17:34,San Francisco Caltrain (Townsend at 4th),70,6/18/2015 17:37,2nd at Townsend,61,370,Subscriber,94107,37.776617,-122.395260,37.780526,-122.390288
3,897674,8/20/2015 7:06,Civic Center BART (7th at Market),72,8/20/2015 7:15,Townsend at 7th,65,451,Subscriber,94582,37.781039,-122.411748,37.771058,-122.402717
4,322830,6/13/2014 8:46,San Francisco Caltrain 2 (330 Townsend),69,6/13/2014 8:57,Embarcadero at Folsom,51,603,Subscriber,95014,37.776600,-122.395470,37.791464,-122.391034
5,487841,10/7/2014 21:41,2nd at Townsend,61,10/7/2014 21:47,Post at Kearny,47,478,Subscriber,94115,37.780526,-122.390288,37.788975,-122.403452
6,677808,3/11/2015 18:09,Market at 10th,67,3/11/2015 18:22,San Francisco Caltrain (Townsend at 4th),70,505,Subscriber,94025,37.776619,-122.417385,37.776617,-122.395260
7,704449,3/30/2015 17:29,Embarcadero at Vallejo,48,3/30/2015 17:35,Steuart at Market,74,356,Subscriber,94536,37.799953,-122.398525,37.794139,-122.394434
8,833587,7/5/2015 11:54,Market at 10th,67,7/5/2015 12:00,Market at 4th,76,401,Subscriber,94102,37.776619,-122.417385,37.786305,-122.404966
9,420411,8/22/2014 13:30,Embarcadero at Vallejo,48,8/22/2014 13:41,5th at Howard,57,363,Subscriber,94114,37.799953,-122.398525,37.781752,-122.405127


In [49]:
trip_train['start_station_lat'] = trip_train[pd.to_numeric(trip_train.start_station_lat, errors='coerce').notnull()]['start_station_lat']
trip_train['start_station_long'] = trip_train[pd.to_numeric(trip_train.start_station_long, errors='coerce').notnull()]['start_station_long']
trip_train['end_station_lat'] = trip_train[pd.to_numeric(trip_train.end_station_lat, errors='coerce').notnull()]['end_station_lat']
trip_train['end_station_long'] = trip_train[pd.to_numeric(trip_train.end_station_long, errors='coerce').notnull()]['end_station_long']


In [51]:
from math import radians, cos, sin, asin, sqrt

#distancia entre dos estaciones 
def distanciaEntreEstaciones(long1, lat1, long2, lat2):
    long1, lat1, long2, lat2 = map(radians, [long1, lat1, long2, lat2])
    dLong = long2 - long1 
    dLat = lat2 - lat1 
    a = sin(dLat/2)**2 + cos(lat1) * cos(lat2) * sin(dLong/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

distancia = trip_train.apply(lambda x: distanciaEntreEstaciones(x[11], x[12], x[13], x[14]), axis = 1)
trip_train['distancia_entre_estaciones'] = distancia


IndexError: ('index out of bounds', 'occurred at index 0')

In [9]:
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,end_station_lat,end_station_long,distancia_entre_estaciones
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,37.795392,-122.394203,37.804770,-122.403234,1.148453
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,37.776619,-122.417385,37.795425,-122.404767,1.794651
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,37.789625,-122.400811,37.785299,-122.396236,0.569924
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,37.776600,-122.395470,37.789756,-122.394643,0.788625
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,37.776619,-122.417385,37.783871,-122.408433,1.084530
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,37.771058,-122.402717,37.776617,-122.395260,0.892320
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,37.782259,-122.392738,37.780526,-122.390288,0.291147
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,37.788446,-122.408499,37.776617,-122.395260,1.631116
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,37.785299,-122.396236,37.776617,-122.395260,0.528155
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,37.786305,-122.404966,37.791300,-122.399051,0.721470


In [10]:
del trip_train['start_station_lat']
del trip_train['start_station_long']
del trip_train['end_station_lat']
del trip_train['end_station_long']

In [11]:
fechasStart = trip_train['start_date'].map(lambda x: (x.split(' ')[0]))
trip_train['startDate'] =  fechasStart
fechasEnd = trip_train['end_date'].map(lambda x: (x.split(' ')[0]))
trip_train['endDate'] =  fechasEnd

In [12]:
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,distancia_entre_estaciones,startDate,endDate
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,1.148453,8/27/2015,8/27/2015
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,1.794651,7/28/2014,7/28/2014
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,0.569924,6/9/2014,6/9/2014
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,0.788625,1/26/2015,1/26/2015
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,1.084530,8/29/2015,8/29/2015
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,0.892320,10/30/2014,10/30/2014
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,0.291147,8/7/2015,8/7/2015
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,1.631116,10/8/2014,10/8/2014
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,0.528155,8/21/2015,8/21/2015
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,0.721470,4/23/2015,4/23/2015


In [13]:
#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['start_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 
trip_train['startDate_weekday'] = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%A"))

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['end_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 
trip_train['endDate_weekday'] = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%A"))

In [14]:
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,distancia_entre_estaciones,startDate,endDate,startDate_weekday,endDate_weekday
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,1.148453,8/27/2015,8/27/2015,Thursday,Thursday
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,1.794651,7/28/2014,7/28/2014,Monday,Monday
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,0.569924,6/9/2014,6/9/2014,Monday,Monday
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,0.788625,1/26/2015,1/26/2015,Monday,Monday
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,1.084530,8/29/2015,8/29/2015,Saturday,Saturday
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,0.892320,10/30/2014,10/30/2014,Thursday,Thursday
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,0.291147,8/7/2015,8/7/2015,Friday,Friday
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,1.631116,10/8/2014,10/8/2014,Wednesday,Wednesday
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,0.528155,8/21/2015,8/21/2015,Friday,Friday
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,0.721470,4/23/2015,4/23/2015,Thursday,Thursday


In [15]:
#separamos en dos campos nuevos. fecha en formato yyyymmdd y hora del dia en minutos
fechasStart = trip_train['start_date'].map(lambda x: (int(dt.datetime.strptime(x.split(' ')[0], "%m/%d/%Y").strftime("%Y%m%d"))))
horasStart = trip_train['start_date'].map(lambda x: (x.split(' ')[1]))
horasStart = horasStart.map(lambda x: (int(x.split(':')[0]) * 60 *60+ int(x.split(':')[1])*60))

fechasEnd = trip_train['end_date'].map(lambda x: (int(dt.datetime.strptime(x.split(' ')[0], "%m/%d/%Y").strftime("%Y%m%d"))))
horasEnd = trip_train['end_date'].map(lambda x: (x.split(' ')[1]))
horasEnd = horasEnd.map(lambda x: (int(x.split(':')[0]) * 60 *60+ int(x.split(':')[1])*60))

In [16]:
#ver dia y noche como afecta

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['start_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

horas = trip_train['start_date'].map(lambda x:x.split('/')[2].split(' ')[1].split(':')[0])
#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 

fechas = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%m"))

i = 0
deNoche = [0] * fechas.size
for line in fechas:
    if(((int(line) < 4) & (int(line) > 0)) | ((int(line) > 9)&(int(line) < 13))):
        if((int(horas[i]) <18) & (int(horas[i]) > 7)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    else:
        if((int(horas[i]) <20) & (int(horas[i]) > 6)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    i = i + 1 
        
trip_train['startDate_deNoche'] = deNoche

#ver dia y noche como afecta

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['end_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

horas = trip_train['end_date'].map(lambda x:x.split('/')[2].split(' ')[1].split(':')[0])
#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 

fechas = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%m"))

i = 0
deNoche = [0] * fechas.size
for line in fechas:
    if(((int(line) < 4) & (int(line) > 0)) | ((int(line) > 9)&(int(line) < 13))):
        if((int(horas[i]) <18) & (int(horas[i]) > 7)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    else:
        if((int(horas[i]) <20) & (int(horas[i]) > 6)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    i = i + 1 
        
trip_train['endDate_deNoche'] = deNoche

In [17]:
weather['zip_code'] = weather['zip_code'].apply(str)


In [18]:
weather[(weather['date'] == '6/9/2014') & (weather['zip_code'] == '94107')]


,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
1020,6/9/2014,78.0,68.0,58.0,56.0,54.0,53.0,80.0,67.0,53.0,...,9.0,7.0,22.0,10.0,25.0,0,1.0,NaN,305.0,94107


In [19]:
trip_train = trip_train.merge(weather, left_on=['startDate', 'zip_code'], right_on= ['date', 'zip_code'] ,how='left')
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,...,10.0,9.0,7.0,22.0,10.0,25.0,0,1.0,NaN,305.0
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,...,10.0,10.0,10.0,16.0,6.0,26.0,0,5.0,NaN,270.0
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,...,10.0,10.0,10.0,22.0,10.0,24.0,0,4.0,NaN,271.0
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,...,10.0,9.0,7.0,16.0,5.0,17.0,0,1.0,NaN,310.0
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
del trip_train['endDate']
del trip_train['startDate']
del trip_train['id']
del trip_train['start_station_name']
del trip_train['end_station_name']
del trip_train['start_date']
del trip_train['end_date']
del trip_train['date']
del trip_train['zip_code']


In [21]:
trip_train['start_date'] = fechasStart
trip_train['end_date'] = fechasEnd
trip_train['start_hour'] = horasStart
trip_train['end_hour'] = horasEnd

In [22]:
#convertimos subscription_type a 1 y 0 
trip_train['duration'] = trip_train[pd.to_numeric(trip_train.duration, errors='coerce').notnull()]['duration']
subscription_type = trip_train['subscription_type'].map(lambda x: 1 if x == 'Subscriber' else 0)
trip_train['subscription_type'] = subscription_type
trip_train.head()

,duration,start_station_id,end_station_id,bike_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour
0,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,20150827,20150827,30960,31380
1,636,67,46,417,1,1.794651,Monday,Monday,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,20140728,20140728,79560,80220
2,334,77,62,281,1,0.569924,Monday,Monday,0,0,...,10.0,25.0,0,1.0,NaN,305.0,20140609,20140609,31320,31620
3,666,69,55,634,1,0.788625,Monday,Monday,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,20150126,20150126,60900,61620
4,318,67,39,607,1,1.084530,Saturday,Saturday,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,20150829,20150829,54540,54840


In [23]:
trip_train['events'].value_counts()

Rain                 8640
Fog                  5415
Fog-Rain              955
Rain-Thunderstorm     176
rain                   38
Name: events, dtype: int64

In [24]:
trip_train['precipitation_inches'] = trip_train['precipitation_inches'].map(lambda x: 0.01 if x == 'T' else x)

In [25]:
#convertimos las clases de la columna events
events = trip_train['events'].map(lambda x: 0 if ((x == 'rain') or (x == 'Rain')) else 1 if x == 'Fog' else 2 if x == 'Fog-Rain' else 3 if x == 'Rain-Thunderstorm' else 4)
trip_train['events'] = events
trip_train.head()

,duration,start_station_id,end_station_id,bike_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour
0,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,NaN,NaN,NaN,NaN,4,NaN,20150827,20150827,30960,31380
1,636,67,46,417,1,1.794651,Monday,Monday,1,1,...,NaN,NaN,NaN,NaN,4,NaN,20140728,20140728,79560,80220
2,334,77,62,281,1,0.569924,Monday,Monday,0,0,...,10.0,25.0,0,1.0,4,305.0,20140609,20140609,31320,31620
3,666,69,55,634,1,0.788625,Monday,Monday,0,0,...,NaN,NaN,NaN,NaN,4,NaN,20150126,20150126,60900,61620
4,318,67,39,607,1,1.084530,Saturday,Saturday,0,0,...,NaN,NaN,NaN,NaN,4,NaN,20150829,20150829,54540,54840


In [26]:
#convertimos las clases de los dias de semana
startDays = trip_train['startDate_weekday'].map(lambda x: 0 if x == 'Monday' else 1 if x == 'Thursday' 
                                             else 2 if x == 'Wednesday' else 3 if x == 'Tuesday' else 4 if x == 'Friday'
                                            else 5 if x == 'Saturday' else 6)
trip_train['startDate_weekday'] = startDays

endDays = trip_train['endDate_weekday'].map(lambda x: 0 if x == 'Monday' else 1 if x == 'Thursday' 
                                             else 2 if x == 'Wednesday' else 3 if x == 'Tuesday' else 4 if x == 'Friday'
                                            else 5 if x == 'Saturday' else 6)
trip_train['endDate_weekday'] = endDays
trip_train['startDate_weekday'].value_counts()

3    100410
2     98407
1     97751
0     95118
4     89915
5     36773
6     31587
Name: startDate_weekday, dtype: int64

In [27]:
trip_train['endDate_weekday'].value_counts()


3    100425
2     98424
1     97756
0     95166
4     89755
5     36741
6     31694
Name: endDate_weekday, dtype: int64

In [28]:
trip_train['events'].value_counts()

4    534737
0      8678
1      5415
2       955
3       176
Name: events, dtype: int64

In [29]:
weather.columns

Index(['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
       'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f',
       'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches',
       'cloud_cover', 'events', 'wind_dir_degrees', 'zip_code'],
      dtype='object')

In [30]:
del trip_train['bike_id']
del trip_train['mean_temperature_f']
del trip_train['min_temperature_f']
del trip_train['max_dew_point_f']
del trip_train['min_dew_point_f']
del trip_train['mean_dew_point_f']
del trip_train['mean_humidity']
del trip_train['min_humidity']
del trip_train['max_sea_level_pressure_inches']
del trip_train['min_sea_level_pressure_inches']
del trip_train['mean_sea_level_pressure_inches']
del trip_train['wind_dir_degrees']
del trip_train['mean_visibility_miles']
del trip_train['min_visibility_miles']
del trip_train['max_wind_Speed_mph']
del trip_train['max_humidity']

In [31]:
trip_train['intervalo'] = trip_train['end_hour'] - trip_train['start_hour']


In [32]:
trip_train = trip_train.drop(['start_date',
       'end_date', 'start_hour', 'end_hour'] ,axis = 1)

In [33]:
q = trip_train
#w = test
q


,duration,start_station_id,end_station_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,max_temperature_f,max_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,intervalo
0,396,50,60,1,1.148453,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4,420
1,636,67,46,1,1.794651,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,660
2,334,77,62,1,0.569924,0,0,0,0,78.0,10.0,10.0,25.0,0,1.0,4,300
3,666,69,55,1,0.788625,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4,720
4,318,67,39,1,1.084530,5,5,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4,300
5,337,65,70,1,0.892320,1,1,1,1,71.0,10.0,6.0,26.0,0,5.0,4,360
6,394,64,61,1,0.291147,4,4,0,0,77.0,10.0,10.0,24.0,0,4.0,4,360
7,766,71,70,1,1.631116,2,2,0,0,75.0,10.0,5.0,17.0,0,1.0,4,780
8,531,62,70,1,0.528155,4,4,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4,540
9,267,76,75,0,0.721470,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,240


In [34]:
q[q['max_temperature_f'].notnull()]

,duration,start_station_id,end_station_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,max_temperature_f,max_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,intervalo
2,334,77,62,1,0.569924,0,0,0,0,78.0,10.0,10.0,25.0,0,1.0,4,300
5,337,65,70,1,0.892320,1,1,1,1,71.0,10.0,6.0,26.0,0,5.0,4,360
6,394,64,61,1,0.291147,4,4,0,0,77.0,10.0,10.0,24.0,0,4.0,4,360
7,766,71,70,1,1.631116,2,2,0,0,75.0,10.0,5.0,17.0,0,1.0,4,780
13,525,69,51,1,1.012936,1,1,0,0,69.0,10.0,7.0,29.0,0,5.0,4,480
14,301,61,70,1,0.599521,0,0,0,0,81.0,10.0,9.0,25.0,0,0.0,4,300
16,528,56,61,1,1.028551,4,4,0,0,72.0,10.0,9.0,22.0,0,3.0,4,540
35,192,49,54,1,0.346678,1,1,1,1,71.0,10.0,6.0,26.0,0,5.0,4,180
43,2408,2,2,1,0.000000,4,4,0,0,67.0,10.0,10.0,36.0,0.01,4.0,0,2400
46,488,50,61,1,0.986144,3,3,0,0,62.0,10.0,5.0,20.0,0,7.0,4,480


In [35]:
a = q['duration'].mean()
b = q['start_station_id'].mean()
c = q['end_station_id'].mean()
d = q['subscription_type'].mean()
e = q['distancia_entre_estaciones'].mean()
f = q['startDate_weekday'].mean()
g = q['endDate_weekday'].mean()
h = q['startDate_deNoche'].mean()
i = q['endDate_deNoche'].mean()
j = q['max_temperature_f'].mean()
k = q['max_visibility_miles'].mean()
l = q['mean_wind_speed_mph'].mean()
m = q['max_gust_speed_mph'].mean()
#n = q['precipitation_inches'].mean()
o = q['cloud_cover'].mean()
p = q['events'].mean()


#ac = q['mean_temperature_f'].mean()
#ad = q['min_temperature_f'].mean()
#ae = q['max_dew_point_f'].mean()
#af = q['min_dew_point_f'].mean()
#ag = q['mean_dew_point_f'].mean()
#ah = q['mean_humidity'].mean()
#ai = q['min_humidity'].mean()
#aj = q['max_sea_level_pressure_inches'].mean()
#ak = q['min_sea_level_pressure_inches'].mean()
#al = q['mean_sea_level_pressure_inches'].mean()
#am = q['wind_dir_degrees'].mean()
#an = q['mean_visibility_miles'].mean()
#ao = q['min_visibility_miles'].mean()
#ap = q['max_wind_Speed_mph'].mean()
#aq = q['max_humidity'].mean()


In [36]:
q1 = q.dropna(how='all')
q1 = q1.replace([np.inf, -np.inf], np.nan)
#q1 = q.fillna(0)
q1['duration'] = q['duration'].fillna(a)
q1['start_station_id'] = q['start_station_id'].fillna(b)
q1['end_station_id'] = q['end_station_id'].fillna(c)
q1['subscription_type'] = q['subscription_type'].fillna(d)
q1['distancia_entre_estaciones'] = q['distancia_entre_estaciones'].fillna(e)
q1['startDate_weekday'] = q['startDate_weekday'].fillna(f)
q1['endDate_weekday'] = q['endDate_weekday'].fillna(g)
q1['startDate_deNoche'] = q['startDate_deNoche'].fillna(h)
q1['endDate_deNoche'] = q['endDate_deNoche'].fillna(i)
q1['max_temperature_f'] = q['max_temperature_f'].fillna(j)
q1['max_visibility_miles'] = q['max_visibility_miles'].fillna(k)
q1['mean_wind_speed_mph'] = q['mean_wind_speed_mph'].fillna(l)
q1['max_gust_speed_mph'] = q['max_gust_speed_mph'].fillna(m)
q1['precipitation_inches'] = q['precipitation_inches'].fillna(0)
q1['cloud_cover'] = q['cloud_cover'].fillna(o)
q1['events'] = q['events'].fillna(p)

#q1 = q1.replace(q1.mean(),np.nan)
#w = q.fillna(0)


#q1['mean_temperature_f'] = q['mean_temperature_f'].fillna(ac)
#q1['min_temperature_f']  = q['min_temperature_f'].fillna(ad)
#q1['max_dew_point_f'] = q['max_dew_point_f'].fillna(ae)
#q1['min_dew_point_f'] = q['min_dew_point_f'].fillna(af)
#q1['mean_dew_point_f'] = q['mean_dew_point_f'].fillna(ag)
#q1['mean_humidity'] = q['mean_humidity'].fillna(ah)
#q1['min_humidity'] = q['min_humidity'].fillna(ai)
#q1['max_sea_level_pressure_inches'] =q['max_sea_level_pressure_inches'].fillna(aj)
#q1['min_sea_level_pressure_inches']  = q['min_sea_level_pressure_inches'].fillna(ak)
#q1['mean_sea_level_pressure_inches'] = q['mean_sea_level_pressure_inches'].fillna(al)
#q1['wind_dir_degrees'] = q['wind_dir_degrees'].fillna(am)
#q1['mean_visibility_miles'] = q['mean_visibility_miles'].fillna(an)
#q1['min_visibility_miles']  = q['min_visibility_miles'].fillna(ao)
#q1['max_wind_Speed_mph'] = q['max_wind_Speed_mph'].fillna(ap)
#q1['max_humidity'] = q['max_humidity'].fillna(aq)

In [206]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

q5 = q1
dur = q5['end_hour'] - q5['start_hour']
q5['dur'] = dur
q5['is_true'] = np.random.uniform(0,1,len(trip_train)) <= 0.1
pruebaD, pruebaT = q5[q5['is_true']==True], q5[q5['is_true']==False]
del q5['is_true']
del pruebaD['is_true']
del pruebaT['is_true']
pruebaD_x = pruebaD.drop('duration'
             , axis=1)
pruebaD_y = pruebaD['duration']
    
pruebaT_x = pruebaT.drop('duration'
             , axis=1)
pruebaT_y = pruebaT['duration']

pruebaD_x = preprocessing.normalize(pruebaD_x)
#pruebaT_x = preprocessing.normalize(pruebaT_x)
#numpyMatrix = pruebaD_x.as_matrix().astype(float)
#pruebaD_x = preprocessing.scale(pruebaD_x)
#pruebaT_x = preprocessing.scale(pruebaT_x)

scaler = StandardScaler()
scaler.fit(pruebaD_x) 
pruebaD_x= scaler.transform(pruebaD_x)
#pruebaD_y = scaler.transform(pruebaD_y) 

In [168]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from numpy import random
from sklearn.svm import LinearSVR
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
import time
#for i in range(50):
print("[INFO] tuning hyperparameters via grid search")
        
params = [
         { 'average' : [False] , 'loss' : ['squared_epsilon_insensitive'],
          'penalty': ['l1', 'l2', 'elasticnet'] ,'learning_rate' : ['invscaling'] , 'eta0' : [1,0.1,0.01,0.0001,0.000000001], 'power_t' : [0.001,0.1,0.45,0.01,0.6],
          'epsilon' : [0.8,0.85,0.1], 'l1_ratio' : [0.4,0.5,0.6,0.8],  'n_iter' : [np.ceil(10**6 / len(pruebaD_x))],
             'alpha' : [1,0.1,0.01,0.001,0.0001,0.0001,0.00001]}  
]
  
#param_dist = {'average' : [True] , 'loss' : ['squared_loss','huber','squared_epsilon_insensitive'],
#              'penalty': ['l1'] ,'learning_rate' : ['optimal', 'invscaling'] , 'eta0' : [ random.uniform(0.0000001, 0.2), random.uniform(0.0000001, 0.2) ,random.uniform(0.0000001, 0.2) ], 'power_t' : [ random.uniform(0.1, 0.6), random.uniform(0.1, 0.6), random.uniform(0.1, 0.6) , random.uniform(0.1, 0.6)],
#             'epsilon' : [random.uniform(0,100), random.uniform(0,100), random.uniform(0,100)], 'l1_ratio' : [random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)], 
#                'alpha' : [random.uniform(0.000001,0.5)]}      
#    
model = linear_model.SGDRegressor()
#grid = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20)

grid = GridSearchCV(model, params, n_jobs = -1)
start = time.time()
print("[INFO] fitting stage")
grid = grid.fit(pruebaD_x, pruebaD_y)
acc = grid.best_score_

print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))
 
#prediction = grid.best_estimator_.predict(pruebaT_x)
#score = mean_squared_error(pruebaT_y, prediction)
#print("[INFO] grid search best parameters predictor score: {}".format((1 - abs(score)) * 100))
    

[INFO] tuning hyperparameters via grid search
[INFO] fitting stage
[INFO] grid search took 2052.62 seconds
[INFO] grid search accuracy: 7.21%
[INFO] grid search best parameters: {'alpha': 1, 'average': False, 'epsilon': 0.8, 'eta0': 0.0001, 'l1_ratio': 0.8, 'learning_rate': 'invscaling', 'loss': 'squared_epsilon_insensitive', 'n_iter': 19.0, 'penalty': 'elasticnet', 'power_t': 0.01}


In [78]:
a = abs(grid.best_estimator_.predict(pruebaD_x))
a[0:20]

array([  358.38686369,   415.63085516,   721.09017981,  1160.08804004,
        4022.26659539,  1291.02542164,    86.54751264,    90.56950698,
        3991.52408146,   583.78615836,   618.75636049,  5819.21590603,
         530.47342908,   538.99846963,   217.02910588,  4655.52704509,
         325.06900221,   431.29681313,  1101.59842594,   890.73148117])

In [ ]:
pruebaD_y[0:20]

In [73]:
sklearn.metrics.mean_absolute_error(a,pruebaD_y)

1178.8633393035411

In [44]:
from sklearn import preprocessing
dur = q1['end_hour'] - q1['start_hour']
q1['dur'] = dur
q5 = q1
q5['is_true'] = np.random.uniform(0,1,len(trip_train)) <= 0.05
pruebaD, pruebaT = q5[q5['is_true']==True], q5[q5['is_true']==False]
del q5['is_true']
del pruebaD['is_true']
del pruebaT['is_true']
pruebaD_x = pruebaD.drop('duration'
             , axis=1)
pruebaD_y = pruebaD['duration']
    
pruebaT_x = pruebaT.drop('duration'
             , axis=1)
pruebaT_y = pruebaT['duration']
pruebaT_y = pruebaT['duration']
pruebaD_x = preprocessing.normalize(pruebaD_x)
#pruebaT_x = preprocessing.normalize(pruebaT_x)
#numpyMatrix = pruebaD_x.as_matrix().astype(float)
#pruebaD_x = preprocessing.scale(numpyMatrix)
#prebaD_x = preprocessing.scale(pruebaD_x)
#pruebaT_x = preprocessing.scale(pruebaT_x)

In [39]:
del q1['mean_wind_speed_mph']
del q1['max_gust_speed_mph']
del q1['precipitation_inches']
del q1['cloud_cover']

In [103]:
q5 = q1

q5

,duration,start_station_id,end_station_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,max_temperature_f,max_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,intervalo
0,396,50,60,1,1.148453,1,1,0,0,69.243171,10.104086,7.558569,23.438198,0,3.570244,4,420
1,636,67,46,1,1.794651,0,0,1,1,69.243171,10.104086,7.558569,23.438198,0,3.570244,4,660
2,334,77,62,1,0.569924,0,0,0,0,78.000000,10.000000,10.000000,25.000000,0,1.000000,4,300
3,666,69,55,1,0.788625,0,0,0,0,69.243171,10.104086,7.558569,23.438198,0,3.570244,4,720
4,318,67,39,1,1.084530,5,5,0,0,69.243171,10.104086,7.558569,23.438198,0,3.570244,4,300
5,337,65,70,1,0.892320,1,1,1,1,71.000000,10.000000,6.000000,26.000000,0,5.000000,4,360
6,394,64,61,1,0.291147,4,4,0,0,77.000000,10.000000,10.000000,24.000000,0,4.000000,4,360
7,766,71,70,1,1.631116,2,2,0,0,75.000000,10.000000,5.000000,17.000000,0,1.000000,4,780
8,531,62,70,1,0.528155,4,4,0,0,69.243171,10.104086,7.558569,23.438198,0,3.570244,4,540
9,267,76,75,0,0.721470,1,1,1,1,69.243171,10.104086,7.558569,23.438198,0,3.570244,4,240


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVR
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn import preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import time

num = 1
max_Accuracy = 0
for i in range(num):
    q5 = q1
    train_x = q5.drop('duration', axis = 1)
    train_y = q5['duration']
    train_x = preprocessing.normalize(train_x)
   
    
    scaler = StandardScaler()
    scaler.fit(train_x) 
    train_x= scaler.transform(train_x)
       
    print("[INFO] tuning hyperparameters via grid search")
        
    params = [
         { 'average' : [False] , 'loss' : ['squared_epsilon_insensitive'],
          'penalty': ['elasticnet'] ,'learning_rate' : ['invscaling'] , 'eta0' : [0.1,1,10,100,0.0001,0.001,0.00001], 'power_t' : [0.45],
          'epsilon' : [0.01,0.4,0.001,0.1,1,10,0.0001,0.00001,0.0000001], 'l1_ratio' : [0.4], 'n_iter' : [15],
             'alpha' : [0.0001]}  
    ]
    
    params = [
         { 'C' : [10,100,1000] , 'epsilon' : [0], 'random_state' : [0], 'loss' : ['epsilon_insensitive' ]}  
    ]
    
    model = LinearSVR()
    #model = linear_model.SGDRegressor()
    grid = GridSearchCV(model, params, n_jobs = -1, cv = 15)
    start = time.time()
    print("[INFO] fitting stage")

    grid = grid.fit(train_x, train_y)
    acc = grid.best_score_
    
    
    #acc = grid.best_score_
    #accs = grid.score(x1, y1)
    print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
    # evaluate the best grid searched model on the testing data
    print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
    print("[INFO] grid search best parameters: {}".format(grid.best_params_))
    #print (accs)
    

[INFO] tuning hyperparameters via grid search
[INFO] fitting stage


In [79]:
prediction = grid.best_estimator_.predict(train_x)
train_y[0:30]

0      396
1      636
2      334
3      666
4      318
5      337
6      394
7      766
8      531
9      267
10     766
11     416
12    4120
13     525
14     301
15     311
16     528
17    4675
18     203
19     916
20     246
21     431
22     722
23     275
24     560
25     697
26    1595
27     470
28     372
29     472
Name: duration, dtype: int64

In [80]:
prediction[0:30]

array([  420.44301355,   706.83882163,   281.05400017,   765.80317202,
         213.89007124,   322.40936237,   334.71654374,   776.13159014,
         586.79590462,   288.87103545,   767.23333738,   409.6986676 ,
        1346.97881655,   504.49043326,   260.28861824,   279.79053409,
         585.77001858,  1365.56514451,    65.78735367,   858.70129994,
         158.5724061 ,   420.86458118,   753.80465664,   197.96827383,
         575.7693444 ,   818.03103739,  1158.18349988,   517.30335106,
         638.91753378,   500.35193425])

In [82]:
def grid_scores_to_df(grid_scores):
    rows = list()
    for grid_score in grid_scores:
        for fold, score in enumerate(grid_score.cv_validation_scores):
            row = grid_score.parameters.copy()
            row['fold'] = fold
            row['score'] = score
            rows.append(row)
    df = pd.DataFrame(rows)
    return df

grid.grid_scores_

M:\Users\Krion\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.12352, std: 0.06104, params: {'C': 10, 'epsilon': 0, 'loss': 'epsilon_insensitive', 'random_state': 0}]

In [83]:
grid.best_estimator_.coef_

array([ -1.32122818e+02,  -1.30373950e+02,  -5.57689802e+00,
        -1.42966719e+01,   5.89734345e+01,  -6.13606254e+01,
        -1.01631511e+00,   5.81449636e-01,  -2.48314793e+02,
        -2.01186160e+02,   3.26390211e+00,  -3.57294617e+01,
         8.82128493e-01,  -5.82968027e+00,  -2.22167117e+00,
        -1.18047410e+03])